In [1]:
from google.protobuf.json_format import MessageToDict
import pandas as pd
import math
import geopandas as gpd
from shapely.geometry import LineString, Point
import tomtomtrafficflowTile_pb2
import os
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# Valores globales
zoom_level = 16
tile = tomtomtrafficflowTile_pb2.Tile()
tiles_largo = 3
tiles_ancho = 3

In [3]:
def tileZXYToLatLon(z, x, y):
    MAX_ZOOM_LEVEL = 22

    if z > MAX_ZOOM_LEVEL or (z is None or not isinstance(z, (int, float))):
        raise ValueError(f"Zoom level value is out of range [0, {MAX_ZOOM_LEVEL}]")
    minXY = 0
    maxXY = int(2**z - 1)
    if x > maxXY or x < minXY or x is None or not isinstance(x, (int, float)):
        raise ValueError(f"Tile x value is out of range [0, {maxXY}]")

    if y > maxXY or y < minXY or y is None or not isinstance(y, (int, float)):
        raise ValueError(f"Tile y value is out of range [0, {maxXY}]")

    
    lon = (x / (2**z)) * 360.0 - 180.0
    n = math.pi - (2.0 * math.pi * y) / (2**z)
    lat = (180.0 / math.pi) * (math.atan(0.5 * (math.exp(n) - math.exp(-n))))
   
    return lat,lon

In [4]:
# Función para convertir coordenadas geodésicas a coordenadas de tiles
def lat_lon_to_tile_zxy(lat, lon, zoom_level):
    
    min_zoom_level = 0
    max_zoom_level = 22
    min_lat = -85.051128779807
    max_lat = 85.051128779806
    min_lon = -180.0
    max_lon = 180.0

    # Check input values of zoom level, latitude and longitude
    if (zoom_level is None or not isinstance(zoom_level, (int, float))
        or zoom_level < min_zoom_level
        or zoom_level > max_zoom_level ):
        
        raise ValueError(
            f"Zoom level value is out of range [{min_zoom_level}, {max_zoom_level}]"
        )

    if lat is None or not isinstance(lat, (int, float)) or lat < min_lat or lat > max_lat:
        raise ValueError(f"Latitude value is out of range [{min_lat}, {max_lat}]")

    if (lon is None or not isinstance(lon, (int, float))
        or lon < min_lon
        or lon > max_lon ):
        
        raise ValueError(f"Longitude value is out of range [{min_lon}, {max_lon}]")

    z = int(zoom_level)
    xy_tiles_count = 2**z
    x = int(((lon + 180.0) / 360.0) * xy_tiles_count)
    y = int(((1.0 - math.log( math.tan((lat * math.pi) / 180.0) + 1.0 / math.cos((lat * math.pi) / 180.0)) /math.pi)/ 2.0)* xy_tiles_count)

    return x,y

In [5]:
def transform(decodedTiles, zoom_level, xTile, yTile):
    lat_inicio, lon_inicio = tileZXYToLatLon(zoom_level, xTile, yTile)
    lat_fin, lon_fin = tileZXYToLatLon(zoom_level, xTile+1, yTile+1)

    # Calcular diferencias en latitud y longitud
    diff_lat =lat_fin-lat_inicio
    diff_lon =lon_fin-lon_inicio

    num_pixeles = 4096
    coordinates = []

    # Calcular incrementos por píxel
    inc_lat = diff_lat / num_pixeles
    inc_lon = diff_lon / num_pixeles
    for x,y in decodedTiles:
        lat_pixel = lat_inicio + (y * inc_lat)
        lon_pixel = lon_inicio + (x * inc_lon)
        coordinates.append((lon_pixel,lat_pixel))
    return coordinates

In [6]:
def decode_geometry(geometry):
    c = 0
    decodeTiles = []
    lines =[]
    decode_x = 0
    decode_y = 0
    while c < len(geometry):
        command_and_count = geometry[c]
        command = command_and_count & 0x7
        count = command_and_count >> 0x3
        #print("comand",command)
        for _ in range(count):
            x = geometry[c+1] 
            y = geometry[c+2]
            decode_x += ((x >> 0x1) ^ (-(x & 0x1))) 
            decode_y += ((y >> 0x1) ^ (-(y & 0x1)))
            if command==1:
                if len(decodeTiles)==0:
                    decodeTiles.append([decode_x, decode_y])    
                else:
                    lines.append(decodeTiles) 
                    decodeTiles = []  
                    decodeTiles.append([decode_x, decode_y])    
            elif command==2:     
                decodeTiles.append([decode_x, decode_y])         
            c += 2
        if command==2:
            lines.append(decodeTiles) 
        c += 1        
        #c += 3
    #return  decodeTiles
    return lines



In [7]:

#Para guardarlo como puntos Point debe ser lat, lon
#Si se guardan como líneas linestring, debe ser lon, lat
def conversion_vector(ruta_archivo, zoom_level, xTile, yTile, fecha, hora, estacion):
  #conversion vector a dataframe
  with open(ruta_archivo, "rb") as f:
    tile.ParseFromString(f.read())
  dict_request = MessageToDict(tile)
  points = []
  dictionary ={}
  if "layers" in dict_request.keys():
    if dict_request['layers'][0]['name'] == 'Traffic flow':
      keys = dict_request['layers'][0]["keys"]
      values = dict_request['layers'][0]["values"]
      for line in dict_request['layers'][0]["features"]:
          all_properties = []
          if line["type"]=="LINESTRING":	
            geometry = line["geometry"]
            lines = decode_geometry(geometry)
            for decodedLines in lines:
                coordinates = transform(decodedLines, zoom_level, xTile, yTile)
                if len(coordinates)>0:
                    tags = line["tags"]
                    all_properties = []
                    for i in range(0, int(len(tags)/2), 2):
                        key = keys[tags[i]]
                        value = values[tags[i+1]]
                        value = value[list(value.keys())[0]]
                        properties = {key:value}
                        all_properties.append(properties)

                    dictionary = {"type": line["type"],
                                "properties": all_properties,
                                "coordinates": coordinates,
                                "date": fecha,
                                "time": hora,
                                "station": estacion}
                    points.append(dictionary)
      # Lista de nombres de columnas
      columnas = ['type', 'properties', 'coordinates',"date","time","station"]
      # Crear un DataFrame a partir de la lista y los nombres de columnas
      dataframe = pd.DataFrame(points, columns=columnas)
  else:
     dataframe = []
     points  = []
  return dataframe, points

In [8]:
ruta_raiz = 'C:/Users/valer/Documents/CIC/doctorado/air_pollution/traffic_flow/vector/vectores/' 
dataframes = []
all_points = []
# Itera sobre todas las carpetas y subcarpetas
for carpeta_actual, carpetas, archivos in os.walk(ruta_raiz):
    # Itera sobre todos los archivos en la carpeta actual
    for nombre_archivo in archivos:
        # Forma la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta_actual, nombre_archivo)
        
        # Verifica si el archivo tiene una extensión específica (por ejemplo, .xlsx)
        if nombre_archivo.endswith('.pbf'):
            # Lee el archivo y agrega el DataFrame a la lista
            nombre = nombre_archivo.split("_")
            zoom_level = int(nombre[2])
            xTile = int(nombre[3])
            yTile = int(nombre[4])
            fecha = nombre[7]+"-"+nombre[6]+"-"+nombre[5]
            ultimo = nombre[10].split(".")
            hora = nombre[8]+":"+nombre[9]+":"+ultimo[0]
            estacion = nombre[0]
            df, points = conversion_vector(ruta_archivo, zoom_level, xTile, yTile, fecha, hora, estacion)
            if len(df)>0:
                dataframes.append(df)
                all_points.append(points)

# Combina todos los DataFrames en uno solo
df_completo = pd.concat(dataframes, ignore_index=True)

# Muestra el DataFrame completo
df_completo

,type,properties,coordinates,date,time,station
0,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.21359449625015, 19.280035796279765), (-9...",01-04-2024,00:00:53,AJM
1,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.21359449625015, 19.280035796279765), (-9...",01-04-2024,00:00:53,AJM
2,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.2120361328125, 19.27939144803043), (-99....",01-04-2024,00:00:53,AJM
3,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.217529296875, 19.275451935747668), (-99....",01-04-2024,00:00:53,AJM
4,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.217529296875, 19.275451935747668), (-99....",01-04-2024,00:00:53,AJM
...,...,...,...,...,...,...
4832758,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08388823270798, 19.521718803216636), (-9...",31-03-2024,09:06:47,XAL
4832759,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08347114920616, 19.52333295275669), (-99...",31-03-2024,09:06:47,XAL
4832760,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL
4832761,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL


In [9]:
df_completo.to_csv("df_completo_2_rows.csv") #4832763 rows × 6 columns

In [15]:
df_completo.to_csv("df_completo_03_abr_24_10322288_rows.csv")

In [14]:
#Para linestring las coordenadas deben tener el orden longitude,latitude
dir_geojson = "geojsons"
stations = df_completo["station"].unique()
for station in stations:        
        values = []
        lines = []
        coverages = []
        fechas = []
        horas = []
        station_name = []
        df = df_completo[df_completo["station"]==station]
        for i in range(0,len(df)):
                values.append(df.iloc[i]["properties"][0]["traffic_level"])
                lines.append(LineString(df.iloc[i]["coordinates"]))
                coverages.append(df.iloc[i]["properties"][1]['traffic_road_coverage'])
                fechas.append(df.iloc[i]["date"])
                horas.append(df.iloc[i]["time"])
                station_name.append(station)

        # Crear un DataFrame de ejemplo con coordenadas y valores
        data = {"traffic_level": values,
                "roadCovrag": coverages,
                "date": fechas,
                "time": horas,
                "Coordinate": lines,
                "station": station_name}

        df = pd.DataFrame(data)

        # Convertir DataFrame a GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='Coordinate')

        name_geojson = "datos_trafico_"+station+'.geojson'
        ruta = os.path.join(dir_geojson,name_geojson)
        # Guardar el GeoDataFrame en un archivo GeoJSON o Shapefile
        gdf.to_file(ruta, driver='GeoJSON')

In [13]:
# Crear GeoJson con todos los datos
#Para linestring las coordenadas deben tener el orden longitude,latitude
 
values = []
lines = []
coverages = []
fechas = []
horas = []
for i in range(0,len(df_completo)):
    values.append(df_completo.iloc[i]["properties"][0]["traffic_level"])
    lines.append(LineString(df_completo.iloc[i]["coordinates"]))
    coverages.append(df_completo.iloc[i]["properties"][1]['traffic_road_coverage'])
    fechas.append(df_completo.iloc[i]["date"])
    horas.append(df_completo.iloc[i]["time"])

# Crear un DataFrame de ejemplo con coordenadas y valores
data = {"traffic_level": values,
        "roadCovrag": coverages,
        "date": fechas,
        "time": horas,
        "Coordinate": lines}

df = pd.DataFrame(data)

# Convertir DataFrame a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='Coordinate')

name_geojson = "all_traffic_data_"+'.geojson'
# Guardar el GeoDataFrame en un archivo GeoJSON o Shapefile
gdf.to_file(name_geojson , driver='GeoJSON')

In [ ]:
#Graficar tiles de un día y hora específico
"""
# Crear una nueva figura
plt.figure()

limites = [0, 0.15, 0.35, 0.75, float('inf')]
colores = ['#777777', '#FF2323', '#FFFF37', '#2BC82B']

for i in range(len(all_points)):
    for j in range(len(all_points[i])):
        if all_points[i][j]["date"]=='27-02-2024' and all_points[i][j]["time"]=="13:31:09":
            latitudes, longitudes = zip(*all_points[i][j]["coordinates"])
            velocidades = all_points[i][j]["properties"][0]["traffic_level"]
            # Mapear las velocidades a los índices de colores
            indices_colores = np.digitize(velocidades, bins=limites, right=True)
            # Graficar las coordenadas en la misma figura
            plt.plot(latitudes, longitudes, color=colores[indices_colores-1], marker='o', linestyle='-')


# Personalizar la apariencia del gráfico
plt.title('Ubicación de Coordenadas en la Calle')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.grid(True)
plt.show()
"""

In [187]:
# Puntos con  las coordenadas bien, solo hay que cambiar la funcion "transform" para que las coordenadas las regrese con el orden lat, lon
"""
values = []
lines = []
coverages = []
fechas = []
horas = []
for i in range(0,len(all_points)):
    for lat, lon in all_points[i][0]["coordinates"]:
        values.append(all_points[i][0]["properties"][0]["traffic_level"])
        lines.append(Point(lon, lat))
        coverages.append(all_points[i][0]["properties"][1]['traffic_road_coverage'])
        fechas.append(all_points[i][0]["date"])
        horas.append(all_points[i][0]["time"])

# Crear un DataFrame de ejemplo con coordenadas y valores
data = {'traffic_level': values,
        "roadCovrag": coverages,
        "date": fechas,
        "time": horas,
        'geometry': lines}

df = pd.DataFrame(data)

# Convertir DataFrame a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)

ruta_geojson = 'conjuntos_coordenadas.geojson'
# Guardar el GeoDataFrame en un archivo GeoJSON o Shapefile
gdf.to_file(ruta_geojson, driver='GeoJSON')
"""

In [1]:
import geopandas as gpd

# Ruta al archivo GeoJSON
ruta_geojson = 'geojsons/datos_trafico_MER.geojson'

# Leer el archivo GeoJSON
datos_geojson = gpd.read_file(ruta_geojson)

# Mostrar los datos leídos
datos_geojson

       traffic_level roadCovrag        date      time station  \
0              0.223       full  23-02-2024  13:56:17     MER   
1              0.252       full  23-02-2024  13:56:17     MER   
2              0.252       full  23-02-2024  13:56:17     MER   
3              0.300       full  23-02-2024  13:56:17     MER   
4              0.300       full  23-02-2024  13:56:17     MER   
...              ...        ...         ...       ...     ...   
19274          1.000   one_side  27-02-2024  23:03:08     MER   
19275          1.000   one_side  27-02-2024  23:03:08     MER   
19276          1.000   one_side  27-02-2024  23:03:08     MER   
19277          1.000   one_side  27-02-2024  23:03:08     MER   
19278          1.000   one_side  27-02-2024  23:03:08     MER   

                                                geometry  
0      LINESTRING (-99.12668 19.42515, -99.12671 19.4...  
1      LINESTRING (-99.12921 19.43033, -99.12826 19.4...  
2      LINESTRING (-99.12887 19.42603, -99

In [10]:
import pandas as pd

# Cargar los DataFrames desde los archivos CSV
df1 = pd.read_csv("df_completo_03_abr_24_10322288_rows.csv")
df2 = pd.read_csv("df_completo_2_rows.csv")

# Unir los DataFrames
df_union = pd.concat([df1, df2], ignore_index=True)

# Si deseas unir los DataFrames basados en una columna común, puedes usar la función merge
# Por ejemplo, si ambos DataFrames tienen una columna 'ID' en común
# df_union = pd.merge(df1, df2, on='ID')

# Guardar el DataFrame resultante en un nuevo archivo CSV
#df_union.to_csv('union.csv', index=False)


In [11]:
df_union

,Unnamed: 0,type,properties,coordinates,date,time,station
0,0,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20654296875, 19.325524613420782), (-99.2...",20-02-2024,18:50:21,PED
1,1,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20379638671875, 19.32602544853046), (-99...",20-02-2024,18:50:21,PED
2,2,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20331090688705, 19.32669491605546), (-99...",20-02-2024,18:50:21,PED
3,3,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
4,4,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
...,...,...,...,...,...,...,...
15155046,4832758,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08388823270798, 19.521718803216636), (-9...",31-03-2024,09:06:47,XAL
15155047,4832759,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08347114920616, 19.52333295275669), (-99...",31-03-2024,09:06:47,XAL
15155048,4832760,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL
15155049,4832761,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL


In [13]:
df_union = df_union.reset_index(drop=True)
df_union

,Unnamed: 0,type,properties,coordinates,date,time,station
0,0,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20654296875, 19.325524613420782), (-99.2...",20-02-2024,18:50:21,PED
1,1,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20379638671875, 19.32602544853046), (-99...",20-02-2024,18:50:21,PED
2,2,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20331090688705, 19.32669491605546), (-99...",20-02-2024,18:50:21,PED
3,3,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
4,4,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
...,...,...,...,...,...,...,...
15155046,4832758,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08388823270798, 19.521718803216636), (-9...",31-03-2024,09:06:47,XAL
15155047,4832759,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08347114920616, 19.52333295275669), (-99...",31-03-2024,09:06:47,XAL
15155048,4832760,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL
15155049,4832761,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL


In [17]:
del df_union["Unnamed: 0"]


In [18]:
df_union

,type,properties,coordinates,date,time,station
0,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20654296875, 19.325524613420782), (-99.2...",20-02-2024,18:50:21,PED
1,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20379638671875, 19.32602544853046), (-99...",20-02-2024,18:50:21,PED
2,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20331090688705, 19.32669491605546), (-99...",20-02-2024,18:50:21,PED
3,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
4,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.20242309570312, 19.325925154955115), (-9...",20-02-2024,18:50:21,PED
...,...,...,...,...,...,...
15155046,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08388823270798, 19.521718803216636), (-9...",31-03-2024,09:06:47,XAL
15155047,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.08347114920616, 19.52333295275669), (-99...",31-03-2024,09:06:47,XAL
15155048,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL
15155049,LINESTRING,"[{'traffic_level': 1.0}, {'traffic_road_covera...","[(-99.07916218042374, 19.52355289169167), (-99...",31-03-2024,09:06:47,XAL


In [19]:
df_union.to_csv("datos_al_07_abr_24_15155051_rows.csv")